Import Libraries

In [177]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import os as os

Import Data

In [178]:
# Function to load the dataset
def load_data(filepath):
    """Load the dataset from a given filepath."""
    return pd.read_csv(filepath)

Preprocess Data

In [179]:
# Function to preprocess the dataset
def preprocess_data(df):
    # Drop null data
    df = df.dropna()

    # Convert Order_Demand to numeric, setting errors='coerce' to convert non-integer values to NaN
    df['Order_Demand'] = pd.to_numeric(df['Order_Demand'], errors='coerce')

    # Drop rows where Order_Demand is NaN (i.e., non-integer values)
    df = df.dropna(subset=['Order_Demand'])

    """Preprocess the data: one-hot encode categorical variables using get_dummies and scale demand."""
    # One-hot encode the categorical columns
    df = pd.get_dummies(df, columns=['Warehouse', 'Product_Category'], drop_first=True)

    # Convert the true and false data into int
    boolean_columns = df.select_dtypes(include=[bool]).columns
    df[boolean_columns] = df[boolean_columns].astype(int)
    
    # Scale the demand values
    scaler = RobustScaler()
    df['Order_Demand'] = scaler.fit_transform(df[['Order_Demand']])
    
    return df

Create Sequences

In [180]:
# Function to create sequences for LSTM
def create_sequences_per_group(data, n_timesteps):
    """Create sequences of demand data, including one-hot encoded variables."""
    X, y = [], []
    
    for i in range(n_timesteps, len(data)):
        # Select the previous 'n_timesteps' rows as input
        X.append(data.iloc[i-n_timesteps:i].values)  # Including one-hot encoded features and demand
        
        # The target is the current 'demand' value at time 'i'
        y.append(data['Order_Demand'].iloc[i])
    
    return np.array(X), np.array(y)

Objective Function

In [181]:
# Objective function for hyperparameter tuning
def objective(space):
    """Objective function to minimize during hyperparameter tuning."""
    lstm_units = space['lstm_units']
    dropout_rate = space['dropout_rate']
    batch_size = space['batch_size']

    # Create the model
    model = create_model((X_train.shape[1], X_train.shape[2]), lstm_units, dropout_rate)

    # Train the model
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=100, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test, verbose=0)
    
    return {'loss': loss, 'status': STATUS_OK}

Prepare Data for LSTM

In [182]:
# Function to prepare the data for LSTM
def prepare_data(df, n_timesteps):
    """Prepare the data for LSTM by preprocessing and creating sequences."""
    df = preprocess_data(df)

    # Drop columns
    df = df.drop(columns = ['Product_Code','Date'])

    X, y = create_sequences_per_group(df, n_timesteps)
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    return X_train, X_test, y_train, y_test

Create the LSTM Model

In [183]:
# Function to create and compile the LSTM model
def create_model(input_shape, lstm_units, dropout_rate):
    """Create and compile the LSTM model."""
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))  # Output layer for demand prediction
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Preprocess Data: Onehot Encoding and Scaler

In [184]:
if __name__ == "__main__":
  # Load your dataset (update the filepath)
    filepath = '/Users/gurmehar/Desktop/Dataset/Historical Product Demand.csv'
    data = load_data(filepath)

    # Hyperparameter space
    space = {
        'lstm_units': hp.choice('lstm_units', [32, 64, 128]),
        'dropout_rate': hp.uniform('dropout_rate', 0.1, 0.5),
        'batch_size': hp.choice('batch_size', [16, 32, 64])
    }

    
    # Set the number of timesteps for the LSTM model
    n_timesteps = 7
    
    # Prepare the data
    X_train, X_test, y_train, y_test = prepare_data(data, n_timesteps)

    # Check the shapes of the resulting datasets
    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)

    # Hyperparameter tuning with Hyperopt
    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)


    print("Best Hyperparameters: ", best)

/var/folders/qb/g5rtpcqs5pg88m10wdwlhljr0000gn/T/ipykernel_4437/4093664349.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Order_Demand'] = pd.to_numeric(df['Order_Demand'], errors='coerce')


X_train shape: (825144, 7, 36)
X_test shape: (206286, 7, 36)
y_train shape: (825144,)
y_test shape: (206286,)
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



  0%|          | 0/10 [31:52<?, ?trial/s, best loss=?]


KeyboardInterrupt: 